In [1]:
import winsound as sd
def beepsound():
    fr = 2000    # range : 37 ~ 32767
    du = 1000     # 1000 ms ==1second
    sd.Beep(fr, du) # winsound.Beep(frequency, duration)
beepsound()

In [2]:
import warnings
warnings.filterwarnings('ignore')
from matplotlib import rc
import matplotlib.pyplot as plt # 한글
rc('font',family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False # 마이너스 부호
import os
import pandas as pd
import seaborn as sns
import numpy as np
from IPython.core.display import HTML
import time
import re

In [ ]:
start = time.time()

path = 'dataset' 
file_list = os.listdir(path)

ldata = {}
keys = []
for file in file_list:
    data = pd.read_csv(f'dataset/{file}',encoding='cp949')
    file_name = file.replace('.txt','')
    ldata[file_name]=data
    keys.append(file_name)

ldata['purprd']['구매일자'] = pd.to_datetime(ldata['purprd']['구매일자'],format='%Y%m%d')
ldata['purprd']['year'] = ldata['purprd']['구매일자'].dt.year
ldata['purprd']['quarter'] = ldata['purprd']['구매일자'].dt.quarter

def 반기(year,quarter):
    if (year==2014) & (quarter in [1,2]):
        return 1
    elif (year==2014) & (quarter in [3,4]):
        return 2
    elif (year==2015) & (quarter in [1,2]):
        return 3
    else:
        return 4

ldata['purprd']['half'] = ldata['purprd'].apply(lambda x: 반기(x['year'],x['quarter']),axis=1)

productcat = pd.read_excel('상품분류1차추가.xlsx')
ldata['purprd'] = ldata['purprd'].merge(productcat[['소분류코드','분석대분류명','분류1']],on='소분류코드')
ldata['purprd'] = ldata['purprd'].merge(ldata['prodcat'][['소분류코드','중분류명','소분류명']], on='소분류코드')
ldata['purprd'].drop(['대분류코드','중분류코드','점포코드','구매시간']
                     ,axis=1,inplace=True)
ldata['purprd'] = ldata['purprd'][ldata['purprd'].제휴사!="D"]
ldata['cust']['거주지역'].fillna(0,inplace=True)

print(time.time()-start)

In [ ]:
ldata['purprd'].rename(columns={'분석대분류명':'대분류명'},inplace=True)

In [ ]:
### 이상치제거 평균금액으로 상품 가격대
# start = time.time()

# li = []
# for i in ldata['purprd']['분석대분류명'].unique():
#     df1=ldata['purprd'].query(f"분석대분류명=='{i}'")[['구매금액']]
#     q1, q3 = np.percentile(df1,[25,75])
#     iqr = q3 - q1
#     upper_outlier = q3+(iqr * 1.5)
#     cat1_mean = round(df1[df1['구매금액']<upper_outlier]['구매금액'].mean())
#     li.append([i,cat1_mean])    
# 대분류평균금액 = pd.DataFrame(li,columns=['분석대분류명','대분류평균금액'])

# li = []
# for x in ldata['purprd']['분석대분류명'].unique():
#     df = ldata['purprd'].query(f"분석대분류명=='{x}'")
#     for y in df['중분류명'].unique():
#         df1 = df.query(f'중분류명=="{y}"')
#         Q1, Q3 = np.percentile(df1['구매금액'],[25,75])
#         if Q1 == Q3:
#             cat2_mean = round(df1['구매금액'].mean())
#             li.append([x,y,cat2_mean])
#         else:
#             IQR = Q3-Q1
#             upper_outlier = round(Q3 + (IQR*1.5))
#             cat2_mean = round(df1[df1['구매금액'] < upper_outlier]['구매금액'].mean())
#             li.append([x,y,cat2_mean])        
# 중분류평균금액 = pd.DataFrame(li,columns=['분석대분류명','중분류명','중분류평균금액'])

# df = pd.merge(중분류평균금액,대분류평균금액,on='분석대분류명')

# def cat_amount(cat1, cat1_mean, cat2_mean):
#     if cat1_mean*2 <= cat2_mean:
#         return "{}고가".format(cat1)
#     elif cat1_mean*0.5 <= cat2_mean:
#         return "{}중가".format(cat1)
#     else:
#         return "{}저가".format(cat1)
    
# df['상품분류'] = df.apply(lambda x: cat_amount(x['분석대분류명'],x['대분류평균금액'],x['중분류평균금액']),axis=1)

# ldata['purprd'] = ldata['purprd'].merge(df[['분석대분류명','중분류명','상품분류']],
#                                        on=['분석대분류명','중분류명'])

# print(time.time()-start)

In [ ]:
### 이상치제거 안한 평균금액!

start = time.time()

li = []
for i in ldata['purprd']['대분류명'].unique():
    cat1_mean = round(ldata['purprd'].query(f"대분류명=='{i}'")['구매금액'].mean())
    li.append([i,cat1_mean])    
대분류평균금액 = pd.DataFrame(li,columns=['대분류명','대분류평균금액'])

li = []
for x in ldata['purprd']['대분류명'].unique():
    df = ldata['purprd'].query(f"대분류명=='{x}'")
    for y in df['중분류명'].unique():
        cat2_mean = round(df.query(f'중분류명=="{y}"')['구매금액'].mean())
        li.append([x,y,cat2_mean])

중분류평균금액 = pd.DataFrame(li,columns=['대분류명','중분류명','중분류평균금액'])

df = pd.merge(중분류평균금액,대분류평균금액,on='대분류명')

def cat_amount(cat1, cat1_mean, cat2_mean):
    if cat1_mean*2 <= cat2_mean:
        return "{}고가".format(cat1)
    elif cat1_mean*0.5 <= cat2_mean:
        return "{}중가".format(cat1)
    else:
        return "{}저가".format(cat1)
    
df['상품분류'] = df.apply(lambda x: cat_amount(x['대분류명'],x['대분류평균금액'],x['중분류평균금액']),axis=1)
ldata['purprd'] = ldata['purprd'].merge(df[['대분류명','중분류명','상품분류']],
                                       on=['대분류명','중분류명'])
print(time.time()-start)

In [ ]:
start = time.time()

def original_cust(df):    
    all_cust= pd.pivot_table(df,
                  index='고객번호',
                  columns='half',
                  values='구매금액',
                  aggfunc='sum')
    original_custom_idx =all_cust.dropna().index.tolist()
    original_custom = df.query(f'고객번호 == {original_custom_idx}')
    return original_custom

기존고객 = original_cust(ldata['purprd'])
기존고객idx = 기존고객['고객번호'].unique().tolist()
print(time.time()-start)

In [ ]:
# 증감율계산 함수
def rate_variation(f1,f2):
    rate_variation = (f2-f1)/f1
    rate_variation.fillna('부정',inplace=True) # Na : 전 후 둘다 0인 경우
    rate_variation.replace({np.inf:'불능'},inplace=True) # inf : 전반기 0인 경우
    return rate_variation

In [ ]:
def variation(f1,f2):
    variation = abs(f2-f1)
    return variation

In [ ]:
# 구매횟수 증감
def p_freq(n1,n2):
    구매횟수 = pd.pivot_table(기존고객.drop_duplicates('영수증번호'),
             index='고객번호',
             columns='half',
             values='영수증번호',
             aggfunc='count').fillna(0)
    구매횟수증감 = rate_variation(구매횟수[n1],구매횟수[n2])
    return pd.Series(구매횟수증감,name=f'rate_p_freq_ratio')

In [ ]:
# 대분류별 구매금액증감
def cat1_p_amount(n1,n2):
    대분류구매금액합계 = pd.pivot_table(기존고객,
              index='고객번호',
             columns=['half','대분류명'],
              values='구매금액',
              aggfunc='sum').fillna(0)
    cat1_p_amount = rate_variation(대분류구매금액합계[n1],대분류구매금액합계[n2])
    for colname in cat1_p_amount.columns:
        cat1_p_amount.rename(columns={colname:f'cat1_p_amount_{colname}_ratio'},inplace=True)
    return cat1_p_amount

In [ ]:
# 대분류별 구매물품수증감
def cat1_p_freq(n1,n2):
    대분류구매물품수합계 = pd.pivot_table(기존고객,
              index='고객번호',
             columns=['half','대분류명'],
              values='구매금액',
              aggfunc='count').fillna(0)
    cat1_p_freq = rate_variation(대분류구매물품수합계[n1],대분류구매물품수합계[n2])
    for colname in cat1_p_freq.columns:
        cat1_p_freq.rename(columns={colname:f'cat1_p_freq_{colname}_ratio'},inplace=True)
    return cat1_p_freq

In [ ]:
# 최대 구매횟수 제휴사(두 반기)
def max_affiliate_p_freq(n1,n2):
    max_구매횟수 = pd.pivot_table(기존고객.drop_duplicates('영수증번호'),
              index='고객번호',
              columns=['half','제휴사'],
              values='영수증번호',
              aggfunc='count')
    max1 = pd.Series(max_구매횟수[n1].transpose().idxmax(),name=n1)
    max2 = pd.Series(max_구매횟수[n2].transpose().idxmax(),name=n2)
    max_affiliate = pd.concat([max1,max2],axis=1)
    max_affiliate['max_affiliate'] = max_affiliate.astype(str).apply('_'.join,axis=1)
    return max_affiliate['max_affiliate']

In [ ]:
# channel 이용유무 : 이용은 1, 이용안했으면 0

def to_binary(x):
    if x>0:
        return 1
    else:
        return 0

def channel():
    channel = pd.pivot_table(ldata['channel'].query(f'고객번호=={기존고객idx}'),
                               index='고객번호',
                               values='이용횟수',
                               aggfunc='sum')
    channel1 = pd.pivot_table(기존고객,
                  index='고객번호',
                  values='제휴사',
                  aggfunc='count').join(channel).fillna(0)[['이용횟수']]    
    return pd.Series(channel1['이용횟수'].apply(lambda x:to_binary(x)),name='channel')

# channel()

In [ ]:
# 반기별 구매금액증감  
def half_inde(df,*tuple_args):
    buy_sum=pd.pivot_table(df,index='고객번호',
              columns='half',
              values=['구매금액'],
              aggfunc={'구매금액':'sum'})
    for i in tuple_args:
        
        buy_sum[f'half_inde'] = (buy_sum[('구매금액',i[1])]-buy_sum[("구매금액",i[0])])/buy_sum[('구매금액',i[0])]
        colname = buy_sum.columns.tolist()[:-1]
        colname.append(f'half_inde_ratio')
        buy_sum.columns = colname
    
    buy_sum.drop([('구매금액', 1),
            ('구매금액', 2),
            ('구매금액', 3),
            ('구매금액', 4)],axis=1,inplace=True)   
        
    return buy_sum

In [ ]:
# 제휴사별 반기별 구매금액증감  
def affiliate_half_inde(df,*tuple_args):
    affiliate_buy_sum=pd.pivot_table(df,index='고객번호',
              columns=['half','제휴사'],
              values=['구매금액'],
              aggfunc={'구매금액':'sum'})   
    
    for i in tuple_args:
        for j in ['A','B','C']:   
            affiliate_buy_sum['제휴사 구매증감'] = (affiliate_buy_sum[('구매금액',i[1],j)]-affiliate_buy_sum[('구매금액',i[0],j)])/affiliate_buy_sum[('구매금액',i[0],j)]
            colname =affiliate_buy_sum.columns.tolist()[:-1]
            colname.append(f'affiliate_half_inde_{j}_ratio')
            affiliate_buy_sum.columns = colname
    affiliate_buy_sum.fillna('부정',inplace=True) # Na : 전 후 둘다 0인 경우
    affiliate_buy_sum.replace({np.inf:'불능'},inplace=True) # inf : 전반기 0인 경우
    affiliate_buy_sum.drop([('구매금액',1, 'A'),
            ('구매금액',1, 'B'),
            ('구매금액',1, 'C'),
            ('구매금액',2, 'A'),
            ('구매금액',2, 'B'),
            ('구매금액',2, 'C'),
            ('구매금액',3, 'A'),
            ('구매금액',3, 'B'),
            ('구매금액',3, 'C'),
            ('구매금액',4, 'A'),
            ('구매금액',4, 'B'),
            ('구매금액',4, 'C'),],axis=1,inplace=True)

    return affiliate_buy_sum

In [ ]:
# 제휴사별 구매횟수증감 
def buy_count(df,*tuple_args):
    count=pd.pivot_table(df.drop_duplicates('영수증번호'),index='고객번호',
              columns=['half','제휴사'],
              values=['영수증번호'],
              aggfunc={'영수증번호':'count'})
    
    for i in tuple_args:

        for j in ['A','B','C']:
            
    
            count['제휴사 구매횟수증감'] = (count[('영수증번호',i[1],j)]-count[('영수증번호',i[0],j)])/count[('영수증번호',i[0],j)]
            colname =count.columns.tolist()[:-1]
            colname.append(f'buy_count_{j}_ratio')
            count.columns = colname
    count.fillna('부정',inplace=True) # Na : 전 후 둘다 0인 경우
    count.replace({np.inf:'불능'},inplace=True) # inf : 전반기 0인 경우
    count.drop([('영수증번호', 1,"A"),
        ('영수증번호', 1,"B"),
        ('영수증번호', 1,"C"),
        ('영수증번호', 2,"A"),
        ('영수증번호', 2,"B"),
        ('영수증번호', 2,"C"),
        ('영수증번호', 3,"A"),
        ('영수증번호', 3,"B"),
        ('영수증번호', 3,"C"),
        ('영수증번호', 4,"A"),
        ('영수증번호', 4,"B"),
        ('영수증번호', 4,"C")],axis=1,inplace=True)   

    return count

In [ ]:
# 일회평균 구매금액 증감
def one_buy_mean_diff(df,*args_tuple):
    buy_sum=pd.pivot_table(df,index='고객번호',
              columns='half',
              values=['구매금액'],
              aggfunc={'구매금액':'sum'})
    
    count=pd.pivot_table(df.drop_duplicates('영수증번호'),index='고객번호',
              columns='half',
              values=['영수증번호'],
              aggfunc={'영수증번호':'count'})
    
    one_buy_mean_diff=buy_sum.join(count)
    
    
    for i in args_tuple:
        cust_first_mean = one_buy_mean_diff[('구매금액',i[0])]/one_buy_mean_diff[('영수증번호', i[0])]
        cust_second_mean = one_buy_mean_diff[('구매금액',i[1])]/one_buy_mean_diff[('영수증번호', i[1])]
        one_buy_mean_diff['구매평균증감'] = (cust_second_mean - cust_first_mean)/cust_first_mean
        colname = one_buy_mean_diff.columns.tolist()[:-1]
        colname.append(f'one_buy_mean_diff_ratio')
        one_buy_mean_diff.columns = colname
    one_buy_mean_diff.drop([( '구매금액', 1),
            ( '구매금액', 2),
            ( '구매금액', 3),
            ( '구매금액', 4),
            ('영수증번호', 1),
            ('영수증번호', 2),
            ('영수증번호', 3),
            ('영수증번호', 4)],axis=1,inplace=True)    
    return one_buy_mean_diff

In [ ]:
# 종속변수
def label(df,half1,half2):
    sale_different = pd.pivot_table(df,index='고객번호',
                                   columns = 'half',
                                   values = '구매금액',
                                   aggfunc= 'sum')
    전체증감율 = (sum(sale_different[int(f'{half2}')])-sum(sale_different[int(f'{half1}')]))/sum(sale_different[int(f'{half1}')])
    sale_different['label'] = (sale_different[int(f'{half2}')] - sale_different[int(f'{half1}')])/sale_different[int(f'{half1}')]/전체증감율
    
    def to_label(증감):
        if 증감>0:
            return 0
        else:
            return 1

    sale_different[f'label'] = sale_different[f'label'].apply(lambda x: to_label(x))
    sale_different = sale_different[[f'label']]
    sale_different.columns = [f'label'] 
    return sale_different

In [ ]:
# 구매횟수 증감폭
def p_freq_wide(n1,n2):
    구매횟수 = pd.pivot_table(기존고객.drop_duplicates('영수증번호'),
             index='고객번호',
             columns='half',
             values='영수증번호',
             aggfunc='count').fillna(0)
    구매횟수증감 = abs(구매횟수[n2]-구매횟수[n1])
    return pd.Series(구매횟수증감,name=f'rate_p_freq_wide')

In [ ]:
# 대분류별 구매금액증감폭
def cat1_p_amount_wide(n1,n2):
    대분류구매금액합계 = pd.pivot_table(기존고객,
              index='고객번호',
             columns=['half','대분류명'],
              values='구매금액',
              aggfunc='sum').fillna(0)
    cat1_p_amount = abs(대분류구매금액합계[n2]-대분류구매금액합계[n1])
    for colname in cat1_p_amount.columns:
        cat1_p_amount.rename(columns={colname:f'cat1_p_amount_{colname}_wide'},inplace=True)
    return cat1_p_amount

In [ ]:
# 대분류별 구매물품수증감
def cat1_p_freq_wide(n1,n2):
    대분류구매물품수합계 = pd.pivot_table(기존고객,
              index='고객번호',
             columns=['half','대분류명'],
              values='구매금액',
              aggfunc='count').fillna(0)
    cat1_p_freq = abs(대분류구매물품수합계[n2]-대분류구매물품수합계[n1])
    for colname in cat1_p_freq.columns:
        cat1_p_freq.rename(columns={colname:f'cat1_p_freq_{colname}_wide'},inplace=True)
    return cat1_p_freq

In [ ]:
# 반기별 구매금액증감폭
def half_inde_wide(df,*tuple_args):
    buy_sum=pd.pivot_table(df,index='고객번호',
              columns='half',
              values=['구매금액'],
              aggfunc={'구매금액':'sum'})
    for i in tuple_args:
        
        buy_sum[f'half_inde{i[1]}_{i[0]}'] = abs(buy_sum[('구매금액',i[1])]-buy_sum[("구매금액",i[0])])
        colname = buy_sum.columns.tolist()[:-1]
        colname.append('half_inde_wide')
        buy_sum.columns = colname
    
    buy_sum.drop([('구매금액', 1),
            ('구매금액', 2),
            ('구매금액', 3),
            ('구매금액', 4)],axis=1,inplace=True)   
        
    return buy_sum

In [24]:
# 제휴사별 반기별 구매금액증감폭
def affiliate_half_inde_wide(df,*tuple_args):
    affiliate_buy_sum=pd.pivot_table(df,index='고객번호',
              columns=['half','제휴사'],
              values=['구매금액'],
              aggfunc={'구매금액':'sum'})   
    
    for i in tuple_args:
        for j in ['A','B','C']:   
            affiliate_buy_sum['제휴사 구매증감'] = abs(affiliate_buy_sum[('구매금액',i[1],j)]-affiliate_buy_sum[('구매금액',i[0],j)])
            colname =affiliate_buy_sum.columns.tolist()[:-1]
            colname.append(f'affiliate_half_inde_{j}_wide')
            affiliate_buy_sum.columns = colname
    affiliate_buy_sum.fillna(0,inplace=True)        
    affiliate_buy_sum.drop([('구매금액',1, 'A'),
            ('구매금액',1, 'B'),
            ('구매금액',1, 'C'),
            ('구매금액',2, 'A'),
            ('구매금액',2, 'B'),
            ('구매금액',2, 'C'),
            ('구매금액',3, 'A'),
            ('구매금액',3, 'B'),
            ('구매금액',3, 'C'),
            ('구매금액',4, 'A'),
            ('구매금액',4, 'B'),
            ('구매금액',4, 'C'),],axis=1,inplace=True)

    return affiliate_buy_sum

In [25]:
# 제휴사별 구매횟수증감폭
def buy_count_wide(df,*tuple_args):
    count=pd.pivot_table(df.drop_duplicates('영수증번호'),index='고객번호',
              columns=['half','제휴사'],
              values=['영수증번호'],
              aggfunc={'영수증번호':'count'})
    
    for i in tuple_args:

        for j in ['A','B','C']:
            
    
            count['제휴사 구매횟수증감'] = abs(count[('영수증번호',i[1],j)]-count[('영수증번호',i[0],j)])
            colname =count.columns.tolist()[:-1]
            colname.append(f'buy_count_{j}_wide')
            count.columns = colname
    count.fillna(0,inplace=True)
    count.drop([('영수증번호', 1,"A"),
        ('영수증번호', 1,"B"),
        ('영수증번호', 1,"C"),
        ('영수증번호', 2,"A"),
        ('영수증번호', 2,"B"),
        ('영수증번호', 2,"C"),
        ('영수증번호', 3,"A"),
        ('영수증번호', 3,"B"),
        ('영수증번호', 3,"C"),
        ('영수증번호', 4,"A"),
        ('영수증번호', 4,"B"),
        ('영수증번호', 4,"C")],axis=1,inplace=True)   

    return count

In [26]:
# 일회평균 구매금액 증감폭
def one_buy_mean_diff_wide(df,*args_tuple):
    buy_sum=pd.pivot_table(df,index='고객번호',
              columns='half',
              values=['구매금액'],
              aggfunc={'구매금액':'sum'})
    
    count=pd.pivot_table(df.drop_duplicates('영수증번호'),index='고객번호',
              columns='half',
              values=['영수증번호'],
              aggfunc={'영수증번호':'count'})
    
    one_buy_mean_diff=buy_sum.join(count)
    
    
    for i in args_tuple:
        cust_first_mean = one_buy_mean_diff[('구매금액',i[0])]/one_buy_mean_diff[('영수증번호', i[0])]
        cust_second_mean = one_buy_mean_diff[('구매금액',i[1])]/one_buy_mean_diff[('영수증번호', i[1])]
        one_buy_mean_diff['구매평균증감'] = abs(cust_second_mean - cust_first_mean)
        colname = one_buy_mean_diff.columns.tolist()[:-1]
        colname.append(f'one_buy_mean_diff_wide')
        one_buy_mean_diff.columns = colname
    one_buy_mean_diff.drop([( '구매금액', 1),
            ( '구매금액', 2),
            ( '구매금액', 3),
            ( '구매금액', 4),
            ('영수증번호', 1),
            ('영수증번호', 2),
            ('영수증번호', 3),
            ('영수증번호', 4)],axis=1,inplace=True)    
    return one_buy_mean_diff

In [27]:
data = pd.read_excel('dataset1011.xlsx',index_col='고객번호')
month = data.iloc[:,14:24]

# <font color=red>__파생변수 추가😀😀__</font>

## 가격대

In [28]:
# 가격대
def price_ratio(n1,n2):

    def 저중고(x):
        if x[-2:] == '저가':
            return '저가'
        elif x[-2:] == '중가':
            return '중가'
        elif x[-2:] == '고가':
            return '고가'

    기존고객['가격대'] = 기존고객['상품분류'].apply(lambda x: 저중고(x))

    가격대 = pd.pivot_table(기존고객,
                        index='고객번호',
                        columns=['half','가격대'],
                        values='구매금액',
                        aggfunc='sum')
    가격대.fillna(0,inplace=True)
    가격대증감율 = rate_variation(가격대[n1],가격대[n2])
    
    for colname in 가격대증감율.columns:
        가격대증감율.rename(columns={colname:f'price_ratio_{colname}'},inplace=True)       
        
    return 가격대증감율

# 가격대증감율(1,2)

In [29]:
# 가격대
def price_wide(n1,n2):

    def 저중고(x):
        if x[-2:] == '저가':
            return '저가'
        elif x[-2:] == '중가':
            return '중가'
        elif x[-2:] == '고가':
            return '고가'

    기존고객['가격대'] = 기존고객['상품분류'].apply(lambda x: 저중고(x))

    가격대 = pd.pivot_table(기존고객,
                        index='고객번호',
                        columns=['half','가격대'],
                        values='구매금액',
                        aggfunc='sum')
    가격대.fillna(0,inplace=True)
    가격대증감폭 = variation(가격대[n1],가격대[n2])    
    
    for colname in 가격대증감폭.columns:
        가격대증감폭.rename(columns={colname:f'price_wide_{colname}'},inplace=True)       
        
    return 가격대증감폭

# 가격대증감폭(1,2)

# 대분류별가격대

In [30]:
def cat1_price_rate(n1,n2):
    대분류가격대 = pd.pivot_table(기존고객,
                        index='고객번호',
                        columns=['half','상품분류'],
                        values='구매금액',
                        aggfunc='sum')
    대분류가격대.fillna(0,inplace=True)
    대분류가격대증감율 = rate_variation(대분류가격대[n1],대분류가격대[n2])
    for colname in 대분류가격대증감율.columns:
        대분류가격대증감율.rename(columns={colname:f'cat1_price_ratio_{colname}'},inplace=True)       
    return 대분류가격대증감율
# cat1_price_rate(1,2)

In [31]:
def cat1_price_wide(n1,n2):
    대분류가격대 = pd.pivot_table(기존고객,
                        index='고객번호',
                        columns=['half','상품분류'],
                        values='구매금액',
                        aggfunc='sum')
    대분류가격대.fillna(0,inplace=True)
    대분류가격대증감폭 = variation(대분류가격대[n1],대분류가격대[n2])
    for colname in 대분류가격대증감폭.columns:
        대분류가격대증감폭.rename(columns={colname:f'cat1_price_wide_{colname}'},inplace=True)       
    return 대분류가격대증감폭.fillna(0)
# cat1_price_wide(1,2)

# 기여율

In [32]:
# 대분류별 기여율
def cont_ratio_cat1(n1,n2):
    cat1 = pd.pivot_table(기존고객,
                  index='고객번호',
                  columns=['half','대분류명'],
                  values='구매금액',
                  aggfunc='sum')

    cat1_1 = cat1[n1].fillna(0)
    cat1_1['sum'] = cat1_1.sum(axis=1)
    cat1_2 = cat1[n2].fillna(0)
    cat1_2['sum'] = cat1_2.sum(axis=1)
    cat1기여율 = cat1_2-cat1_1

    for i in 기존고객['대분류명'].unique():
        cat1기여율[i] = cat1기여율[i].div(cat1기여율['sum'])*100
    for colname in cat1기여율.columns:
        cat1기여율.rename(columns={colname:f'cont_ratio_cat1{colname}'},inplace=True)

    return cat1기여율.drop('cont_ratio_cat1sum',axis=1)

# cont_ratio_cat1(1,2)

In [33]:
def cont_ratio_affiliate(n1,n2):
    제휴사 = pd.pivot_table(기존고객,
                  index='고객번호',
                  columns=['half','제휴사'],
                  values='구매금액',
                  aggfunc='sum')

    제휴사1 = 제휴사[n1].fillna(0)
    제휴사1['sum'] = 제휴사1.sum(axis=1)
    제휴사2 = 제휴사[n2].fillna(0)
    제휴사2['sum'] = 제휴사2.sum(axis=1)
    제휴사기여율 = 제휴사2-제휴사1

    for i in ['A','B','C']:
        제휴사기여율[i] = 제휴사기여율[i].div(제휴사기여율['sum'])*100
    for colname in 제휴사기여율.columns:
        제휴사기여율.rename(columns={colname:f'cont_ratio_affiliate{colname}'},inplace=True)

    return 제휴사기여율.drop('cont_ratio_affiliatesum',axis=1).fillna(0)

# cont_ratio_affiliate(1,2)

# 추가(즁현)

In [34]:
def class1_p_amount(n1,n2):
    분류1구매금액합계 = pd.pivot_table(기존고객,
              index='고객번호',
             columns=['half','분류1'],
              values='구매금액',
              aggfunc='sum').fillna(0)
    class1_p_amount = rate_variation(분류1구매금액합계[n1],분류1구매금액합계[n2])
    for colname in class1_p_amount.columns:
        class1_p_amount.rename(columns={colname:f'class1_p_amount_{colname}_ratio'},inplace=True)
    return class1_p_amount

In [35]:
def class1_p_freq(n1,n2):
    분류1구매물품수합계 = pd.pivot_table(기존고객,
              index='고객번호',
             columns=['half','분류1'],
              values='구매금액',
              aggfunc='count').fillna(0)
    class1_p_freq = rate_variation(분류1구매물품수합계[n1],분류1구매물품수합계[n2])
    for colname in class1_p_freq.columns:
        class1_p_freq.rename(columns={colname:f'class1_p_freq_{colname}_ratio'},inplace=True)
    return class1_p_freq

In [36]:
def class1_p_amount_wide(n1,n2):
    분류1구매금액합계 = pd.pivot_table(기존고객,
              index='고객번호',
             columns=['half','분류1'],
              values='구매금액',
              aggfunc='sum').fillna(0)
    class1_p_amount = abs(분류1구매금액합계[n2]-분류1구매금액합계[n1])
    for colname in class1_p_amount.columns:
        class1_p_amount.rename(columns={colname:f'class1_p_amount_{colname}_wide'},inplace=True)
    return class1_p_amount

In [37]:
def class1_p_freq_wide(n1,n2):
    분류1구매물품수합계 = pd.pivot_table(기존고객,
              index='고객번호',
             columns=['half','분류1'],
              values='구매금액',
              aggfunc='count').fillna(0)
    class1_p_freq = abs(분류1구매물품수합계[n2]-분류1구매물품수합계[n1])
    for colname in class1_p_freq.columns:
        class1_p_freq.rename(columns={colname:f'class1_p_freq_{colname}_wide'},inplace=True)
    return class1_p_freq

In [38]:
def class1_p_freq_net(n1,n2):
    분류1구매물품수합계 = pd.pivot_table(기존고객,
              index='고객번호',
             columns=['half','분류1'],
              values='구매금액',
              aggfunc='count').fillna(0)
    class1_p_freq = 분류1구매물품수합계[n2]-분류1구매물품수합계[n1]
    for colname in class1_p_freq.columns:
        class1_p_freq.rename(columns={colname:f'class1_p_freq_{colname}_net'},inplace=True)
    return class1_p_freq

In [39]:
def class1_p_amount_net(n1,n2):
    분류1구매금액합계 = pd.pivot_table(기존고객,
              index='고객번호',
             columns=['half','분류1'],
              values='구매금액',
              aggfunc='sum').fillna(0)
    class1_p_amount = 분류1구매금액합계[n2]-분류1구매금액합계[n1]
    for colname in class1_p_amount.columns:
        class1_p_amount.rename(columns={colname:f'class1_p_amount_{colname}_net'},inplace=True)
    return class1_p_amount

# <font color=red>__데이터셋😀😀__</font>

In [41]:
##### 참고용

def make_dataset(df,n1,n2,n3):
    dataset = pd.DataFrame(p_freq(n1,n2)).join([cat1_p_amount(n1,n2), # 반기별구매횟수증감율, 대분류별구매금액증감율
                                   cat1_p_freq(n1,n2), # 대분류별 구매물품수증감율
                                   max_affiliate_p_freq(n1,n2), # 최대구매금액제휴사 변동
                                   channel(), # 채널이용유무
                                   half_inde(df,(n1,n2)), # 반기별구매금액증감율
                                   affiliate_half_inde(df,(n1,n2)), # 제휴사별구매금액증감율
                                   buy_count(df,(n1,n2)), # 반기별구매횟수증감율
                                   one_buy_mean_diff(df,(n1,n2)), # 1회평균구매금액증감율
                                                
                                   ldata['cust'].set_index('고객번호'), # 성별, 연령대, 거주지역
                                                
                                   p_freq_wide(n1,n2), # 구매횟수변동폭
                                   cat1_p_amount_wide(n1,n2), # 대분류별구매금액변동폭
                                   cat1_p_freq_wide(n1,n2), # 대분류별구매물품수변동폭
                                   half_inde_wide(df,(n1,n2)), # 반기별구매금액변동폭
                                   affiliate_half_inde_wide(df,(n1,n2)), # 제휴사별구매금액변동폭
                                   buy_count_wide(df,(n1,n2)), # 반기별구매횟수변동폭
                                   one_buy_mean_diff_wide(df,(n1,n2)), # 1회평균구매금액변동폭
                                                
                                   month, # 월별 구매횟수 증감율
                                   price_ratio(n1,n2), # 가격대구매금액증감율
                                   price_wide(n1,n2), # 가격대구매금액변동폭
                                   cat1_price_rate(n1,n2), # 대분류별가격대구매금액증감율
                                   cat1_price_wide(n1,n2), # 대분류별가격대구매금액변동폭
                                   cont_ratio_cat1(n1,n2), # 대분류별구매금액기여율
                                   cont_ratio_affiliate(n1,n2), # 제휴사별구매금액기여율
                                                
                                   class1_p_amount(n1,n2), # 분류1 구매금액 증감율
                                   class1_p_freq(n1,n2), # 분류1 구매횟수 증감율
                                   class1_p_amount_wide(n1,n2), # 분류1 구매금액 변동폭
                                   class1_p_freq_wide(n1,n2), # 분류1 구매횟수 변동폭
                                   class1_p_freq_net(n1,n2), # 분류1 구매횟수 증감
                                   class1_p_amount_net(n1,n2), # 분류1 구매금액 증감
                                   label(df,n1,n3)]) # 종속변수
    return dataset

dataset1_2 = make_dataset(기존고객,1,2,3)
dataset2_3 = make_dataset(기존고객,2,3,4)

In [42]:
##### 수정(최종이길..)

def make_dataset(df,n1,n2,n3):
    dataset = pd.DataFrame(half_inde(df,(n1,n2))).join([ldata['cust'].set_index('고객번호').drop('거주지역',axis=1), # 반기별구매금액증감율 # 성별, 연령대
                                   p_freq_wide(n1,n2), # 구매횟수변동폭
                                   half_inde_wide(df,(n1,n2)), # 반기별구매금액변동폭
                                   cat1_p_freq_wide(n1,n2), # 대분류별구매물품수변동폭
                                   affiliate_half_inde_wide(df,(n1,n2)), # 제휴사별구매금액변동폭
                                   one_buy_mean_diff_wide(df,(n1,n2)), # 1회평균구매금액변동폭
                                                        
                                   price_ratio(n1,n2), # 가격대구매금액증감율
                                   price_wide(n1,n2), # 가격대구매금액변동폭
                                   cat1_price_rate(n1,n2), # 대분류별가격대구매금액증감율
                                   cat1_price_wide(n1,n2), # 대분류별가격대구매금액변동폭
                                   cont_ratio_cat1(n1,n2), # 대분류별구매금액기여율
                                   cont_ratio_affiliate(n1,n2), # 제휴사별구매금액기여율
                                                
                                   class1_p_amount(n1,n2), # 분류1 구매금액 증감율
                                   class1_p_freq(n1,n2), # 분류1 구매횟수 증감율
                                   class1_p_amount_wide(n1,n2), # 분류1 구매금액 변동폭
                                   class1_p_freq_wide(n1,n2), # 분류1 구매횟수 변동폭
                                   class1_p_freq_net(n1,n2), # 분류1 구매횟수 증감
                                   class1_p_amount_net(n1,n2), # 분류1 구매금액 증감
                                   label(df,n1,n3)]) # 종속변수
    return dataset

please_last1_2 = make_dataset(기존고객,1,2,3)
please_last2_3 = make_dataset(기존고객,2,3,4)

In [43]:
#### 카테고리 전
dataset1_2.to_excel('1016/dataset1_2.xlsx')
dataset2_3.to_excel('1016/dataset2_3.xlsx')
please_last1_2.to_excel('1016/please_last1_2.xlsx')
please_last2_3.to_excel('1016/please_last2_3.xlsx')

In [44]:
beepsound()

In [45]:
dataset1_2 = pd.read_excel('dataset1_2.xlsx',index_col='고객번호')
dataset2_3 = pd.read_excel('dataset2_3.xlsx',index_col='고객번호')

please_last1_2 = pd.read_excel('please_last1_2.xlsx',index_col='고객번호')
please_last2_3 = pd.read_excel('please_last2_3.xlsx',index_col='고객번호')

# category

In [88]:
dataset1_2_cat = dataset1_2.copy()
dataset2_3_cat = dataset2_3.copy()

please_last1_2_cat = please_last1_2.copy()
please_last2_3_cat = please_last2_3.copy()

In [89]:
dataset2_3_cat.drop(['cat1_price_ratio_의약저가','cat1_price_ratio_의약중가','cat1_price_ratio_의약고가'],
                   axis=1,inplace=True)
dataset1_2_cat.drop(['cat1_price_ratio_의약저가','cat1_price_ratio_의약중가','cat1_price_ratio_의약고가'],
                   axis=1,inplace=True)
please_last1_2_cat.drop(['cat1_price_ratio_의약저가','cat1_price_ratio_의약중가','cat1_price_ratio_의약고가'],
                   axis=1,inplace=True)
please_last2_3_cat.drop(['cat1_price_ratio_의약저가','cat1_price_ratio_의약중가','cat1_price_ratio_의약고가'],
                   axis=1,inplace=True)

In [90]:
display(dataset1_2_cat.head(1), please_last1_2_cat.head(10))

,rate_p_freq_ratio,cat1_p_amount_가공식품_ratio,cat1_p_amount_교육문화_ratio,cat1_p_amount_기타_ratio,cat1_p_amount_디지털_ratio,cat1_p_amount_미용품_ratio,cat1_p_amount_스포츠레저_ratio,cat1_p_amount_신선식품_ratio,cat1_p_amount_의류_ratio,cat1_p_amount_의약_ratio,cat1_p_amount_인테리어_ratio,cat1_p_amount_일상용품_ratio,cat1_p_amount_패션잡화_ratio,cat1_p_freq_가공식품_ratio,cat1_p_freq_교육문화_ratio,cat1_p_freq_기타_ratio,cat1_p_freq_디지털_ratio,cat1_p_freq_미용품_ratio,cat1_p_freq_스포츠레저_ratio,cat1_p_freq_신선식품_ratio,cat1_p_freq_의류_ratio,cat1_p_freq_의약_ratio,cat1_p_freq_인테리어_ratio,cat1_p_freq_일상용품_ratio,cat1_p_freq_패션잡화_ratio,max_affiliate,channel,half_inde_ratio,affiliate_half_inde_A_ratio,affiliate_half_inde_B_ratio,affiliate_half_inde_C_ratio,buy_count_A_ratio,buy_count_B_ratio,buy_count_C_ratio,one_buy_mean_diff_ratio,성별,연령대,거주지역,rate_p_freq_wide,cat1_p_amount_가공식품_wide,cat1_p_amount_교육문화_wide,cat1_p_amount_기타_wide,cat1_p_amount_디지털_wide,cat1_p_amount_미용품_wide,cat1_p_amount_스포츠레저_wide,cat1_p_amount_신선식품_wide,...,cat1_price_wide_의약고가,cat1_price_wide_의약저가,cat1_price_wide_의약중가,cat1_price_wide_인테리어고가,cat1_price_wide_인테리어저가,cat1_price_wide_인테리어중가,cat1_price_wide_일상용품고가,cat1_price_wide_일상용품저가,cat1_price_wide_일상용품중가,cat1_price_wide_패션잡화고가,cat1_price_wide_패션잡화저가,cat1_price_wide_패션잡화중가,cont_ratio_cat1가공식품,cont_ratio_cat1교육문화,cont_ratio_cat1기타,cont_ratio_cat1디지털,cont_ratio_cat1미용품,cont_ratio_cat1스포츠레저,cont_ratio_cat1신선식품,cont_ratio_cat1의류,cont_ratio_cat1의약,cont_ratio_cat1인테리어,cont_ratio_cat1일상용품,cont_ratio_cat1패션잡화,cont_ratio_affiliateA,cont_ratio_affiliateB,cont_ratio_affiliateC,class1_p_amount_선매품_ratio,class1_p_amount_전문품_ratio,class1_p_amount_편의품_ratio,class1_p_freq_선매품_ratio,class1_p_freq_전문품_ratio,class1_p_freq_편의품_ratio,class1_p_amount_선매품_wide,class1_p_amount_전문품_wide,class1_p_amount_편의품_wide,class1_p_freq_선매품_wide,class1_p_freq_전문품_wide,class1_p_freq_편의품_wide,class1_p_freq_선매품_net,class1_p_freq_전문품_net,class1_p_freq_편의품_net,class1_p_amount_선매품_net,class1_p_amount_전문품_net,class1_p_amount_편의품_net,label
고객번호,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.377358,-0.485089,부정,불능,불능,-0.302163,-1,0.231797,0.211893,불능,25.8519,5.726069,-0.082938,0.48,부정,불능,불능,-0.05,-1,0.75,0.205128,불능,11,2.6,-0.388889,A_A,0,0.712206,0.692125,16.254128,0.995917,0.337662,3,1.25,0.243108,M,60세이상,60,60,1852600,0,13000,9062100,510550,2077650,220187,...,0,0,28500,0,0,4592590,2905180,26820,43380,236550,276920,661300,-13.054793,0.0,0.091608,63.858273,-3.597714,-14.640662,1.551601,13.303049,0.200832,32.362793,20.966733,-1.04172,96.097043,2.496942,1.406015,1.028575,부정,0.125234,0.176471,부정,0.623529,13317040,0,873917,12,0,106,12,0,106,13317040,0,873917,1


,half_inde_ratio,성별,연령대,rate_p_freq_wide,half_inde_wide,cat1_p_freq_가공식품_wide,cat1_p_freq_교육문화_wide,cat1_p_freq_기타_wide,cat1_p_freq_디지털_wide,cat1_p_freq_미용품_wide,cat1_p_freq_스포츠레저_wide,cat1_p_freq_신선식품_wide,cat1_p_freq_의류_wide,cat1_p_freq_의약_wide,cat1_p_freq_인테리어_wide,cat1_p_freq_일상용품_wide,cat1_p_freq_패션잡화_wide,affiliate_half_inde_A_wide,affiliate_half_inde_B_wide,affiliate_half_inde_C_wide,one_buy_mean_diff_wide,price_ratio_고가,price_ratio_저가,price_ratio_중가,price_wide_고가,price_wide_저가,price_wide_중가,cat1_price_ratio_가공식품고가,cat1_price_ratio_가공식품저가,cat1_price_ratio_가공식품중가,cat1_price_ratio_교육문화고가,cat1_price_ratio_교육문화저가,cat1_price_ratio_교육문화중가,cat1_price_ratio_기타고가,cat1_price_ratio_기타저가,cat1_price_ratio_기타중가,cat1_price_ratio_디지털고가,cat1_price_ratio_디지털저가,cat1_price_ratio_디지털중가,cat1_price_ratio_미용품고가,cat1_price_ratio_미용품저가,cat1_price_ratio_미용품중가,cat1_price_ratio_스포츠레저저가,cat1_price_ratio_스포츠레저중가,cat1_price_ratio_신선식품고가,cat1_price_ratio_신선식품저가,...,cat1_price_wide_의약고가,cat1_price_wide_의약저가,cat1_price_wide_의약중가,cat1_price_wide_인테리어고가,cat1_price_wide_인테리어저가,cat1_price_wide_인테리어중가,cat1_price_wide_일상용품고가,cat1_price_wide_일상용품저가,cat1_price_wide_일상용품중가,cat1_price_wide_패션잡화고가,cat1_price_wide_패션잡화저가,cat1_price_wide_패션잡화중가,cont_ratio_cat1가공식품,cont_ratio_cat1교육문화,cont_ratio_cat1기타,cont_ratio_cat1디지털,cont_ratio_cat1미용품,cont_ratio_cat1스포츠레저,cont_ratio_cat1신선식품,cont_ratio_cat1의류,cont_ratio_cat1의약,cont_ratio_cat1인테리어,cont_ratio_cat1일상용품,cont_ratio_cat1패션잡화,cont_ratio_affiliateA,cont_ratio_affiliateB,cont_ratio_affiliateC,class1_p_amount_선매품_ratio,class1_p_amount_전문품_ratio,class1_p_amount_편의품_ratio,class1_p_freq_선매품_ratio,class1_p_freq_전문품_ratio,class1_p_freq_편의품_ratio,class1_p_amount_선매품_wide,class1_p_amount_전문품_wide,class1_p_amount_편의품_wide,class1_p_freq_선매품_wide,class1_p_freq_전문품_wide,class1_p_freq_편의품_wide,class1_p_freq_선매품_net,class1_p_freq_전문품_net,class1_p_freq_편의품_net,class1_p_amount_선매품_net,class1_p_amount_전문품_net,class1_p_amount_편의품_net,label
고객번호,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.712206,M,60세이상,60,14190957,48,0,1,10,1,10,30,8,2,11,26,7,13637090,354340,199527,30465.548462,1.03018,0.230997,0.543849,7750437,224670,6215850,-0.886999,불능,0.844579,부정,부정,부정,부정,부정,부정,불능,부정,불능,-0.344184,불능,불능,부정,-1,-0.353013,불능,...,0,0,28500,0,0,4592590,2905180,26820,43380,236550,276920,661300,-13.054793,0.000000,0.091608,63.858273,-3.597714,-14.640662,1.551601,13.303049,0.200832,32.362793,20.966733,-1.041720,96.097043,2.496942,1.406015,1.028575,부정,0.125234,0.176471,부정,0.623529,13317040,0,873917,12,0,106,12,0,106,13317040,0,873917,1
2,-0.098718,M,60세이상,21,2401130,15,0,1,2,9,2,36,7,0,6,3,9,2417080,15950,0,1891.176902,-0.17334,0.594819,-0.092427,1854420,616780,1163490,-0.3419,부정,-0.149183,부정,부정,부정,부정,부정,부정,부정,부정,-0.516072,-0.442518,부정,-0.614187,부정,-0.289732,-0.681066,부정,...,0,0,0,0,0,491700,2525920,500,101260,434150,34410,461690,15.087480,0.000000,-0.083294,30.556446,144.830142,45.952947,54.640523,-110.175209,0.000000,20.477858,-101.000779,-0.286115,100.664271,-0.664271,0.000000,-0.010764,불능,-0.193975,-0.240741,불능,-0.196491,110620,434150,2724660,13,1,56,-13,1,-56,-110620,434150,-2724660,0
3,0.224783,M,60세이상,40,191339,26,0,0,1,1,0,3,1,0,0,13,0,0,0,264379,444.438616,1.25656,0.263538,0.152839,43100,55666,92573,1.165354,0.00711,0.036557,부정,부정,부정,부정,부정,부정,부정,불능,부정,부정,-0.177238,부정,부정,부정,1.235955,0.29652,...,0,0,0,0,0,0,2500,12970,74490,0,0,0,23.251925,0.000000,0.000000,14.058817,-1.562671,0.000000,59.046509,-41.810608,0.000000,0.000000,47.016029,0.000000,-41.810608,3.637523,138.173085,-0.66375,부정,0.316953,0,부정,0.123919,53100,0,244439,0,0,43,0,0,43,-53100,0,244439,1
4,-0.155609,F,60세이상,32,784596,58,0,0,0,1,3,37,9,1,3,2,2,916740,340680,208536,3744.020855,0.049041,-0.109242,-0.216455,43716,71450,756862,2.125,-0.196311,-0.451609,부정,부정,부정,부정,부정,부정,부정,부정,부정,16.315789,-1,-0.391892,부정,3.355882,-0.877405,0.743347,...,0,0,10520,0,12700,404000,109400,2760,13840,0,202550,498520,62.622037,0.000000,0.000000,0.000000,-65.422205,

In [91]:
# 카테고리 대상 컬럼들  

# 전체
t_to_cat_columns =list(dataset1_2_cat.columns[dataset1_2_cat.columns.str.contains('ratio')])
t_to_cat_wide_columns = list(dataset1_2_cat.columns[dataset1_2_cat.columns.str.contains('wide')])
t_to_cat_net_columns = list(dataset1_2_cat.columns[dataset1_2_cat.columns.str.contains('net')])
t_le_columns = [17,27,28,29]

# 골라낸거(please_last)
to_cat_columns = list(please_last1_2_cat.columns[please_last1_2_cat.columns.str.contains('ratio')])
to_cat_wide_columns = list(please_last1_2_cat.columns[please_last1_2_cat.columns.str.contains('ratio')])
to_cat_net_columns =list(please_last1_2_cat.columns[please_last1_2_cat.columns.str.contains('net')])
le_columns = [1,2]

In [92]:
def to_cat(df, col,n=4):
    data = df[col]
    data_float = data[(data!='불능')&(data!='부정')].astype(float)
    if len(data_float) != 0:
        data_minus = data_float[data_float<=0]
        data_minus_qcut = pd.cut(data_minus,n,labels=list(range(1,n+1)))

        data_plus = data_float[(0<data_float)&(data_float<=1)]
        data_plus_qcut = pd.cut(data_plus,n,labels=list(range(1+n,n*2+1)))

        data_float[1<data_float] = 2*n+1
        data_outlier = data_float[1<data_float]

        data_object = data[(data=='불능')|(data=='부정')]
        data_object.replace({'부정':0,'불능':2*n+2},inplace=True)
        return pd.concat([data_minus_qcut,data_plus_qcut,data_outlier,data_object])
    
    else:
        data_object = data[(data=='불능')|(data=='부정')]
        data_object.replace({'부정':0,'불능':1},inplace=True)
        return data_object

for i in t_to_cat_columns:
    dataset1_2_cat[i] = to_cat(dataset1_2_cat,i)
for i in t_to_cat_columns:
    dataset2_3_cat[i] = to_cat(dataset2_3_cat,i)

for i in to_cat_columns:
    please_last1_2_cat[i] = to_cat(please_last1_2_cat, i)
for i in to_cat_columns:
    please_last2_3_cat[i] = to_cat(please_last2_3_cat, i)

In [93]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for i in t_le_columns:
    dataset1_2_cat[dataset1_2_cat.columns[i]] = le.fit_transform(dataset1_2_cat[dataset1_2_cat.columns[i]])
for i in t_le_columns:
    dataset2_3_cat[dataset2_3_cat.columns[i]] = le.fit_transform(dataset2_3_cat[dataset2_3_cat.columns[i]])
    
for i in le_columns:
    please_last1_2_cat[please_last1_2_cat.columns[i]] = le.fit_transform(please_last1_2_cat[please_last1_2_cat.columns[i]])
for i in le_columns:
    please_last2_3_cat[please_last2_3_cat.columns[i]] = le.fit_transform(please_last2_3_cat[please_last2_3_cat.columns[i]])

In [95]:
def to_cat_wide(df,col,n=4):
    data = df[col]
    q1, q3 = np.percentile(df[col],[25,75])    
    iqr=q3 - q1    
    lower_bound = q1-(iqr * 1.5)
    upper_bound = q3+(iqr * 1.5)

    def deny(x):
        if x >upper_bound:
            return "up"
        elif x < lower_bound:
            return "lo"
        else:
            return x

    data = df[col].apply(deny)
    data_float = data[(data!='lo')&(data!='up')].astype(float)

    if len(data_float) != 0:
        data_float_cut = pd.cut(data_float,n,labels=list(range(1,n+1)))

        data_object = data[(data=='lo')|(data=='up')]
        data_object.replace({'up':n+1,'lo':0},inplace=True)

        return pd.concat([data_float_cut,data_object])
        
    else:
        return df[col].fillna(0)

for i in t_to_cat_wide_columns:
    dataset1_2_cat[i] = to_cat_wide(dataset1_2_cat,i)
for i in t_to_cat_wide_columns:
    dataset2_3_cat[i] = to_cat_wide(dataset2_3_cat,i)
for i in to_cat_wide_columns:
    please_last1_2_cat[i] = to_cat_wide(please_last1_2_cat,i)
for i in to_cat_wide_columns:
    please_last2_3_cat[i] = to_cat_wide(please_last2_3_cat,i)

In [96]:
def to_cat_net(df,col,n=4):
    data = df[col]
    q1, q3 = np.percentile(df[col],[25,75])    
    iqr=q3 - q1    
    lower_bound = q1-(iqr * 1.5)
    upper_bound = q3+(iqr * 1.5)

    def deny(x):
        if x >upper_bound:
            return "up"
        elif x < lower_bound:
            return "lo"
        else:
            return x

    data = df[col].apply(deny)
    data_float = data[(data!='lo')&(data!='up')].astype(float)

    if len(data_float) != 0:
        data_float_cut = pd.cut(data_float,n+1,labels=list(range(1,n+2)))

        data_object = data[(data=='lo')|(data=='up')]
        data_object.replace({'up':n+2,'lo':0},inplace=True)

        return pd.concat([data_float_cut,data_object])
        
    else:
        return df[col].fillna(0)

In [97]:
for i in t_to_cat_net_columns:
    dataset1_2_cat[i] = to_cat_net(dataset1_2_cat,i)
for i in t_to_cat_net_columns:
    dataset2_3_cat[i] = to_cat_net(dataset2_3_cat,i)
for i in to_cat_net_columns:
    please_last1_2_cat[i] = to_cat_net(please_last1_2_cat,i)
for i in to_cat_net_columns:
    please_last2_3_cat[i] = to_cat_net(please_last2_3_cat,i)

In [98]:
#### 카테고리 후
dataset1_2_cat.to_excel('1015/dataset1_2_cat.xlsx')
dataset2_3_cat.to_excel('1015/dataset2_3_cat.xlsx')
please_last1_2_cat.to_excel('1015/please_last1_2_cat.xlsx')
please_last2_3_cat.to_excel('1015/please_last2_3_cat.xlsx')